In [1]:
import socket


In [2]:
HOST = "192.168.219.154"
PORT = 7770
BUF_SIZE = 65535

In [3]:
pair = {}

In [4]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((HOST, PORT))

In [5]:
# init db connection
import psycopg2
from datetime import datetime

conn = psycopg2.connect(host='192.168.219.105', dbname='chat', user='postgres', password='ces@hufs')
cur = conn.cursor()

In [ ]:
print('Listening at', sock.getsockname())

while True:
    try:
        inData, inAddr = sock.recvfrom(BUF_SIZE)
        keys = list(pair.keys())
        if not inAddr in keys:
            print('{} connected'.format(inAddr))
            pair[inAddr] = inAddr # at start I'm my own partner
            keys = list(pair.keys())
        
        outAddr = pair[inAddr] 
        
        # make pair of chatter in case of no partner
        if inAddr == outAddr:
            #print('inAddr={}'.format(inAddr))
            keys.remove(inAddr)
            
            # find the first no partner addr
            anew = None
            for addr in keys:
                atemp = pair[addr]
                #print('atemp={} addr={}'.format(atemp, addr))
                if addr == atemp:
                    anew = atemp
                    break
                    
            if not anew is None:
                #print('anew={}'.format(anew))
                outAddr = anew
                pair[inAddr] = outAddr
                pair[outAddr] = inAddr       
        
        sock.sendto(inData, outAddr)
        try:
            cur.execute("""
            INSERT INTO chat_server_table 
            (msg_date, host_from, port_from, host_to, port_to, msg_data)
            VALUES (%s, %s, %s, %s, %s, %s)
            """, (datetime.now(), inAddr[0], inAddr[1], outAddr[0], outAddr[1], inData.decode('utf-8')))
        except psycopg2.Error as e:
            print (e.pgerror)
        conn.commit()

    except KeyboardInterrupt:
        print("Exited by user")
        break

cur.close()
conn.close()

Listening at ('192.168.219.154', 7770)
('192.168.219.154', 54055) connected
('192.168.219.172', 52382) connected
('192.168.219.172', 60158) connected
('192.168.219.172', 60159) connected
